In [1]:

import sys
!{sys.executable} -m pip install google-cloud-bigquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 32.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.8/183.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 4.3 MB/s eta 0:00:00


Set dataset file path

In [65]:
file_path = 'gs://schiphol-flight-data-bucket/flight_data/2023-10-04.csv'
file_path_dests = 'gs://schiphol-flight-data-bucket/destination_data/destinations_with_coords.csv'

Create sparksession and load flight data

In [66]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder \
    .appName("FlightBatchProcessing") \
    .getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Load the flight data from your source (e.g., CSV, Parquet, etc.)
flight_data = spark.read \
    .format("csv") \
    .option("header", "true") \
    .load(file_path)


Count the number of incoming and outgoing flights for the entire day

In [67]:
# Count the number of arrival and departure flights
flight_counts = flight_data.groupBy("flightDirection", "scheduleDate").count()

# Show the results
flight_counts.show()

+---------------+------------+-----+
|flightDirection|scheduleDate|count|
+---------------+------------+-----+
|              D|  2023-10-04| 2466|
|              A|  2023-10-04| 2434|
+---------------+------------+-----+



Calculate the average difference between the actual and estimated landing time 

In [68]:
from  pyspark.sql.functions import abs

# Convert timestamp columns to Spark TimestampType
flight_data = flight_data \
            .withColumn("actualLandingTime", col("actualLandingTime") \
            .cast("timestamp")) \
            .withColumn("estimatedLandingTime", col("estimatedLandingTime") \
            .cast("timestamp"))

# Calculate the offset between estimated and actual landing times
landing_time_offset = flight_data \
    .withColumn("landingTimeOffset", abs(col("actualLandingTime").cast("long") - col("estimatedLandingTime").cast("long")))  # Convert seconds to minutes

# Calculate the average offset
average_offset = landing_time_offset.groupBy("scheduleDate").agg({"landingTimeOffset": "avg"})

# Show the results
average_offset.show()


+------------+----------------------+
|scheduleDate|avg(landingTimeOffset)|
+------------+----------------------+
|  2023-10-04|    10.772425249169435|
+------------+----------------------+



Check gate and pier utilization

In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when


# Replace null or NaN values with a placeholder value (e.g., "Unknown")
flight_data = flight_data.na.fill("Unknown")

# Filter only departure flights
departure_data = flight_data.filter(col("flightDirection") == "D")

# Group by gate and pier, and count the number of departure flights for each
gate_counts = departure_data.groupBy("gate", "scheduleDate").count().withColumnRenamed("count", "gateCount")
pier_counts = departure_data.groupBy("pier", "scheduleDate").count().withColumnRenamed("count", "pierCount")

# Calculate the total number of non-null departure flights
total_departure_flights = departure_data.na.drop(subset=["gate", "pier"]).count()

# Calculate the percentage of gate utilization for departures
gate_utilization = gate_counts.withColumn("gatePercentage", (col("gateCount") / total_departure_flights) * 100)

# Calculate the percentage of pier utilization for departures
pier_utilization = pier_counts.withColumn("pierPercentage", (col("pierCount") / total_departure_flights) * 100)

# Show the results
print("Departure Gate Utilization:")
gate_table = gate_utilization.select("gate", "gatePercentage", "scheduleDate")

gate_table.show(truncate=False)

print("Departure Pier Utilization:")
pier_table = pier_utilization.select("pier", "pierPercentage", "scheduleDate")

pier_table.show(truncate=False)


Departure Gate Utilization:
+----+-------------------+------------+
|gate|gatePercentage     |scheduleDate|
+----+-------------------+------------+
|C10 |0.8921330089213302 |2023-10-04  |
|D6  |10.908353609083536 |2023-10-04  |
|M1  |0.16220600162206003|2023-10-04  |
|D29 |0.9732360097323601 |2023-10-04  |
|D24 |0.36496350364963503|2023-10-04  |
|C15 |1.094890510948905  |2023-10-04  |
|D2  |0.48661800486618007|2023-10-04  |
|E9  |0.4460665044606651 |2023-10-04  |
|D79 |0.5677210056772101 |2023-10-04  |
|B3  |0.8110300081103    |2023-10-04  |
|C18 |1.2165450121654502 |2023-10-04  |
|B36 |0.9732360097323601 |2023-10-04  |
|B24 |1.5409570154095702 |2023-10-04  |
|D84 |1.257096512570965  |2023-10-04  |
|B34 |1.7437145174371453 |2023-10-04  |
|C5  |2.4330900243309004 |2023-10-04  |
|D59 |0.527169505271695  |2023-10-04  |
|D10 |0.24330900243309003|2023-10-04  |
|B27 |1.05433901054339   |2023-10-04  |
|M3  |0.28386050283860503|2023-10-04  |
+----+-------------------+------------+
only showing

Check the ratio of charter and passenger flights

In [70]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Replace null or NaN values with a placeholder value (e.g., "Unknown")
flight_data = flight_data.na.fill("Unknown")

# Filter only departure flights
departure_data = flight_data.filter(col("flightDirection") == "D")

# Group by service type and count the number of departure flights for each
service_type_counts = departure_data.groupBy("serviceType", "scheduleDate").count().withColumnRenamed("count", "serviceTypeCount")

# Calculate the total number of departure flights
total_departure_flights = departure_data.count()

# Calculate the percentage of service type utilization for departures
service_type_percentage = service_type_counts.withColumn("percentage", (col("serviceTypeCount") / total_departure_flights) * 100)

# Show the results
print("Service Type Utilization for Departures:")
service_type_table = service_type_percentage.select("serviceType", "percentage", "scheduleDate")
service_type_table.show(truncate=False)


Service Type Utilization for Departures:
+-----------+-------------------+------------+
|serviceType|percentage         |scheduleDate|
+-----------+-------------------+------------+
|F          |1.1354420113544201 |2023-10-04  |
|C          |0.48661800486618007|2023-10-04  |
|H          |0.04055150040551501|2023-10-04  |
|J          |98.29683698296837  |2023-10-04  |
|P          |0.04055150040551501|2023-10-04  |
+-----------+-------------------+------------+



Define a udf so that in the next code block we can change the string representation of the route dict into a dict again

In [71]:
from pyspark.sql.types import StringType, MapType, ArrayType, StructType, StructField, BooleanType
from pyspark.sql.functions import udf

# Define a UDF to parse the string and convert it to a dictionary
def string_to_dict_or_list(s):
    import ast
    return ast.literal_eval(s)

# Register the UDF
udf_string_to_dict = udf(string_to_dict_or_list, MapType(StringType(), StringType()))
udf_string_to_list = udf(string_to_dict_or_list, ArrayType(StringType()))


Check for the top 10 destinations

In [72]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, from_json, split

# Filter only departure flights
departure_data = flight_data.filter(col("flightDirection") == "D")

# Convert the string representation of the route to a dictionary
departure_data = departure_data.withColumn("route_dict", udf_string_to_dict(departure_data["route"]))

# Split the destinations string into an array
departure_data = departure_data.withColumn("destinations_array", split(col("route_dict.destinations"), ","))

# Explode the destinations array to have one row per destination
destinations_data = departure_data.select("destinations_array", "scheduleDate").withColumn("destination", explode("destinations_array"))

# Group by destination and count the number of departure flights for each
destination_counts = destinations_data.groupBy("destination", "scheduleDate").count().withColumnRenamed("count", "flightCount")

# Sort the destinations by flight count in descending order
sorted_destinations = destination_counts.orderBy(col("flightCount").desc())

# Take the top 10 destinations
top_10_destinations = sorted_destinations.limit(10)

# Show the results
print("Top 10 Destinations for Departures:")
top_10_destinations.show(truncate=False)


Top 10 Destinations for Departures:
+-----------+------------+-----------+
|destination|scheduleDate|flightCount|
+-----------+------------+-----------+
|[CDG]      |2023-10-04  |98         |
|[MAN]      |2023-10-04  |73         |
|[LHR]      |2023-10-04  |72         |
|[CPH]      |2023-10-04  |70         |
|[ARN]      |2023-10-04  |59         |
|[OSL]      |2023-10-04  |51         |
|[MAD]      |2023-10-04  |50         |
|[BER]      |2023-10-04  |48         |
|[BCN]      |2023-10-04  |48         |
|[MUC]      |2023-10-04  |46         |
+-----------+------------+-----------+



Load destinations data in order to be sent to bigquery

In [73]:
# Load the flight data from your source (e.g., CSV, Parquet, etc.)
destinations_data = spark.read \
    .format("csv") \
    .option("header", "true") \
    .load(file_path_dests)

destinations_data = destinations_data.select("iata", "longitude", "latitude")

destinations_data.show()

+----+------------------+----------+
|iata|         longitude|  latitude|
+----+------------------+----------+
| AAC|        33.8032762| 31.132093|
| AAE|7.7500122000000005| 36.897375|
| AAF|       -84.9832435|29.7257675|
| AAH| 6.083886800000001|50.7753455|
| AAL|          9.921747|57.0488195|
| AAN|        55.8023118|24.1301619|
| AAP|       -95.3698028|29.7604267|
| AAQ|        37.3158041|44.8935914|
| AAR|         10.203921| 56.162939|
| ABA|        91.4293172|53.7175644|
| ABC|         -1.860173|38.9942576|
| ABD|        48.2754711|30.3666414|
| ABE|       -75.4714098|40.6022939|
| ABI|-99.73314390000002|32.4487364|
| ABJ|        -4.0082563| 5.3599517|
| ABQ|       -106.650422|35.0843859|
| ABR|        -2.0937528|57.1498891|
| ABS|        31.6156242|22.3460086|
| ABT|       106.2479843|-6.9330665|
| ABU|       124.8987786|-9.1064894|
+----+------------------+----------+
only showing top 20 rows



In [74]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, abs

# Filter only departure flights
departure_data = flight_data.filter(col("flightDirection") == "D")

# Convert the string representation of the route to a dictionary
departure_data = departure_data.withColumn("route_dict", udf_string_to_dict(departure_data["route"]))

# Create a new column 'new_column' with the desired values
departure_data = departure_data.withColumn('destinations', col('route_dict').getItem('destinations'))

departure_data = departure_data.withColumn("destinations_list", udf_string_to_list(departure_data["destinations"]))

departure_data.select("destinations").show()



+------------+
|destinations|
+------------+
|       [LEJ]|
|       [ZTH]|
|       [RAK]|
|       [RAK]|
|       [ALC]|
|       [ALC]|
|       [KLX]|
|       [BCN]|
|       [OPO]|
|       [OPO]|
|       [CFU]|
|       [TFS]|
|       [TFS]|
|       [ZTH]|
|       [OLB]|
|       [OLB]|
|       [FNC]|
|       [FNC]|
|       [FNC]|
|       [TFS]|
+------------+
only showing top 20 rows



In [75]:

# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "tmp-bucket-for-data-engineering"  # use your bucket 
spark.conf.set('temporaryGcsBucket', bucket)

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Saving the data to BigQuery
flight_counts.write.format('bigquery') \
  .option('table', 'schiphol_data.flight_counts') \
  .mode("append") \
  .save()

top_10_destinations.write.format('bigquery') \
  .option('table', 'schiphol_data.popular_dests') \
  .mode("append") \
  .save()

gate_table.write.format('bigquery') \
  .option('table', 'schiphol_data.gate_utilization') \
  .mode("append") \
  .save()

pier_table.write.format('bigquery') \
  .option('table', 'schiphol_data.pier_utilization') \
  .mode("append") \
  .save()

service_type_table.write.format('bigquery') \
  .option('table', 'schiphol_data.service_type_utilization') \
  .mode("append") \
  .save()

destinations_data.write.format('bigquery') \
  .option('table', 'schiphol_data.destinations_data') \
  .mode("overwrite") \
  .save()

In [110]:
spark.stop()